In [ ]:
!pip install -q SPARQLWrapper

In [ ]:
!pip install -e /dss/dsshome1/04/di93zer/git/cellnet --no-deps

In [ ]:
import os
from os.path import join

import pandas as pd
import numpy as np

In [ ]:
DATA_PATH = '/mnt/dssmcmlfs01/merlin_cxg_2023_05_15_sf-log1p'

# Compute lookup matrices 

In [ ]:
cell_type_mapping = pd.read_parquet(join(DATA_PATH, 'categorical_lookup/cell_type.parquet'))

inverse_mapping = (
    cell_type_mapping
    .assign(idx=range(len(cell_type_mapping)))
    .set_index('label', drop=True)
)
inverse_mapping.head()

In [ ]:
from cellnet.utils.cell_ontology import retrieve_child_nodes_from_ubergraph


celltypes = cell_type_mapping.label.tolist()
child_nodes_dict = {}
for k, v in retrieve_child_nodes_from_ubergraph(celltypes).items():
    child_nodes_dict[k] = [elem for elem in v if elem in celltypes]

In [ ]:
children_idx = []

for cell_type in cell_type_mapping.label:
    child_nodes = child_nodes_dict[cell_type]
    children_idx.append(inverse_mapping.loc[child_nodes].idx.sort_values().tolist())

cell_type_mapping['children'] = children_idx

In [ ]:
os.makedirs(join(DATA_PATH, 'cell_type_hierarchy'), exist_ok=True)

In [ ]:
child_matrix = np.eye(len(cell_type_mapping))

for i, child_nodes in enumerate(cell_type_mapping.children):
    child_matrix[i, child_nodes] = 1.
    
with open(join(DATA_PATH, 'cell_type_hierarchy/child_matrix.npy'), 'wb') as f:
    np.save(f, child_matrix)

# Sanity check lookup matrices

In [ ]:
cell_type_mapping.loc[np.where(child_matrix[0, :] == 1.)[0]].label.tolist()